Ссылка на БД: https://tochno.st/datasets/bdmo


In [292]:
%%capture
!pip install osmnx pyrosm
!apt-get update -qq && apt-get install -y osmium-tool

In [293]:
import pandas as pd
import numpy as np
import json
from pyrosm import OSM, get_data
import requests
from shapely.geometry import LineString
from shapely.ops import polygonize, unary_union
import json
import matplotlib.pyplot as plt

In [294]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Население (данные 2023 года)



In [295]:
# Площадь каждого округа в гектарах (спарсил ручками)
districts_area = {

    # Адмиралтейский
    "Коломна": 1.89,
    "Сенной округ": 1.10,
    "Адмиралтейский округ": 2.10,
    "Семеновский": 1.66,
    "Измайловское": 3.19,
    "Екатерингофский": 3.05,

    # Василеостровский
    "№ 7": 1.33,
    "Васильевский": 2.41,
    "Гавань": 3.20,
    "Морской": 1.80,
    "Остров Декабристов": 6.28,

    # Выборгский
    "Сампсониевское": 4.90,
    "Светлановское": 22.03,
    "Сосновское": 5.77,
    "№ 15": 4.20,
    "Сергиевское": 6.13,
    "Шувалово-Озерки": 7.20,
    "Левашово": 34.43,
    "Парголово": 42.00,

    # Калининский
    "Гражданка": 4.15,
    "Академическое": 6.57,
    "Финляндский округ": 10.21,
    "№ 21": 3.98,
    "Пискаревка": 9.07,
    "Северный": 2.15,
    "Прометей": 4.05,

    # Кировский
    "Княжево": 4.55,
    "Ульянка": 4.81,
    "Дачное": 4.30,
    "Автово": 12.00,
    "Нарвский округ": 7.61,
    "Красненькая речка": 2.03,
    "Морские ворота": 0.79,

    # Колпинский
    "Колпино": 57.49,
    "Понтонный": 8.00,
    "Усть-Ижора": 4.15,
    "Петро-Славянка": 17.32,
    "Сапёрный": 4.00,
    "Металлострой": 1.12,

    # Красногвардейский
    "Полюстрово": 20.85,
    "Большая Охта": 5.80,
    "Малая Охта": 5.30,
    "Пороховые": 7.13,
    "Ржевка": 17.09,

    # Красносельский
    "Юго-Запад": 2.84,
    "Южно-Приморский": 10.28,
    "Сосновая поляна": 8.87,
    "Урицк": 4.81,
    "Константиновское": 13.09,
    "Горелово": 33.00,
    "Красное Село": 19.28,

    # Кронштадт
    "Кронштадт":19.35,

    # Курортный
    "Зеленогорск": 40.00,
    "Сестрорецк": 88.40,
    "Белоостров": 25.04,
    "Комарово": 31.00,
    "Молодежное": 12.46,
    "Песочный": 17.60,
    "Репино": 15.77,
    "Серово": 9.78,
    "Смолячково": 8.35,
    "Солнечное": 15.57,
    "Ушково": 25.00,

    # Московский
    "Московская застава": 10.01,
    "Гагаринское": 7.97,
    "Новоизмайловское": 6.69,
    "Пулковский меридиан": 35.91,
    "Звездное": 8.90,

    # Невский
    "Невская застава": 7.11,
    "Ивановский": 5.49,
    "Обуховский": 5.51,
    "Рыбацкое": 9.16,
    "Народный": 9.86,
    "№ 54": 6.11,
    "Невский округ": 4.44,
    "Оккервиль": 2.49,
    "Правобережный": 6.40,

    # Петроградский
    "Введенский": 1.07,
    "Кронверкское": 1.74,
    "Посадский": 1.74,
    "Аптекарский остров": 2.57,
    "округ Петровский": 2.92,
    "Чкаловское": 9.50,

    # Петродворцовый
    "Ломоносов": 41.00,
    "Петергоф": 48.30,
    "Стрельна": 19.42,

    # Приморский
    "Лахта-Ольгино": 31.00,
    "№ 65": 11.16,
    "Ланское": 7.84,
    "Комендантский аэродром": 5.45,
    "Озеро Долгое": 5.50,
    "Юнтолово": 11.96,
    "Коломяги": 26.41,
    "Лисий Нос": 8.53,

    # Пушкинский
    "Павловск": 39.80,
    "Пушкин": 89.30,
    "Шушары": 106.63,
    "Александровская": 4.77,
    "Тярлево": 2.62,

    # Фрунзенский
    "Волковское": 12.54,
    "№ 72": 3.6,
    "Купчино": 2.82,
    "Георгиевский": 4.2,
    "Александровский": 10.0,
    "Балканский": 4.24,

    # Центральный
    "Дворцовый округ": 1.75,
    "№ 78": 1.31,
    "Литейный округ": 2.36,
    "Смольнинское": 5.44,
    "Лиговка-Ямская": 3.30,
    "Владимирский округ": 2.43,
}

In [296]:
districts_age = {

    # Адмиралтейский
    "Коломна": 42.44,
    "Сенной округ": 42.44,
    "Адмиралтейский округ": 42.44,
    "Семеновский": 42.44,
    "Измайловское": 42.44,
    "Екатерингофский": 42.44,

    # Василеостровский
    "№ 7": 43.25,
    "Васильевский": 43.25,
    "Гавань": 43.25,
    "Морской": 43.25,
    "Остров Декабристов": 43.25,

    # Выборгский
    "Сампсониевское": 41.95,
    "Светлановское": 41.95,
    "Сосновское": 41.95,
    "№ 15": 41.95,
    "Сергиевское": 41.95,
    "Шувалово-Озерки": 41.95,
    "Левашово": 41.95,
    "Парголово": 41.95,

    # Калининский
    "Гражданка": 43.05,
    "Академическое": 43.05,
    "Финляндский округ": 43.05,
    "№ 21": 43.05,
    "Пискаревка": 43.05,
    "Северный": 43.05,
    "Прометей": 43.05,

    # Кировский
    "Княжево": 43.39,
    "Ульянка": 43.39,
    "Дачное": 43.39,
    "Автово": 43.39,
    "Нарвский округ": 43.39,
    "Красненькая речка": 43.39,
    "Морские ворота": 43.39,

    # Колпинский
    "Колпино": 43.05,
    "Понтонный": 43.05,
    "Усть-Ижора": 43.05,
    "Петро-Славянка": 43.05,
    "Сапёрный": 43.05,
    "Металлострой": 43.05,

    # Красногвардейский
    "Полюстрово": 42.14,
    "Большая Охта": 42.14,
    "Малая Охта": 42.14,
    "Пороховые": 42.14,
    "Ржевка": 42.14,

    # Красносельский
    "Юго-Запад": 41.53,
    "Южно-Приморский": 41.53,
    "Сосновая поляна": 41.53,
    "Урицк": 41.53,
    "Константиновское": 41.53,
    "Горелово": 41.53,
    "Красное Село": 41.53,

    # Кронштадт
    "Кронштадт": 41.13,

    # Курортный
    "Зеленогорск": 42.78,
    "Сестрорецк": 42.78,
    "Белоостров": 42.78,
    "Комарово": 42.78,
    "Молодежное": 42.78,
    "Песочный": 42.78,
    "Репино": 42.78,
    "Серово": 42.78,
    "Смолячково": 42.78,
    "Солнечное": 42.78,
    "Ушково": 42.78,

    # Московский
    "Московская застава": 42.77,
    "Гагаринское": 42.77,
    "Новоизмайловское": 42.77,
    "Пулковский меридиан": 42.77,
    "Звездное": 42.77,

    # Невский
    "Невская застава": 41.55,
    "Ивановский": 41.55,
    "Обуховский": 41.55,
    "Рыбацкое": 41.55,
    "Народный": 41.55,
    "№ 54": 41.55,
    "Невский округ": 41.55,
    "Оккервиль": 41.55,
    "Правобережный": 41.55,

    # Петроградский
    "Введенский": 41.14,
    "Кронверкское": 41.14,
    "Посадский": 41.14,
    "Аптекарский остров": 41.14,
    "округ Петровский": 41.14,
    "Чкаловское": 41.14,

    # Петродворцовый
    "Ломоносов": 42.22,
    "Петергоф": 42.22,
    "Стрельна": 42.22,

    # Приморский
    "Лахта-Ольгино": 41.15,
    "№ 65": 41.15,
    "Ланское": 41.15,
    "Комендантский аэродром": 41.15,
    "Озеро Долгое": 41.15,
    "Юнтолово": 41.15,
    "Коломяги": 41.15,
    "Лисий Нос": 41.15,

    # Пушкинский
    "Павловск": 39.07,
    "Пушкин": 39.07,
    "Шушары": 39.07,
    "Александровская": 39.07,
    "Тярлево": 39.07,

    # Фрунзенский
    "Волковское": 43.53,
    "№ 72": 43.53,
    "Купчино": 43.53,
    "Георгиевский": 43.53,
    "Александровский": 43.53,
    "Балканский": 43.53,

    # Центральный
    "Дворцовый округ": 43.95,
    "№ 78": 43.95,
    "Литейный округ": 43.95,
    "Смольнинское": 43.95,
    "Лиговка-Ямская": 43.95,
    "Владимирский округ": 43.95,
}

In [297]:
# Площадь каждого округа в гектарах (спарсил ручками)
districts_pop = {

    # Адмиралтейский
    "Коломна": 33543,
    "Сенной округ": 20342,
    "Адмиралтейский округ": 23971,
    "Семеновский": 22109,
    "Измайловское": 32761,
    "Екатерингофский": 21698,

    # Василеостровский
    "№ 7": 36749,
    "Васильевский": 31937,
    "Гавань": 41719,
    "Морской": 36676,
    "Остров Декабристов": 61639,

    # Выборгский
    "Сампсониевское": 39654,
    "Светлановское": 90456,
    "Сосновское": 60831,
    "№ 15": 61428,
    "Сергиевское": 60398,
    "Шувалово-Озерки": 117262,
    "Левашово": 6134,
    "Парголово": 115762,

    # Калининский
    "Гражданка": 67736,
    "Академическое": 108955,
    "Финляндский округ": 72500,
    "№ 21": 76698,
    "Пискаревка": 82129,
    "Северный": 48225,
    "Прометей": 76729,

    # Кировский
    "Княжево": 59414,
    "Ульянка": 75768,
    "Дачное": 70009,
    "Автово": 42496,
    "Нарвский округ": 33237,
    "Красненькая речка": 38080,
    "Морские ворота": 10415,

    # Колпинский
    "Колпино": 147179,
    "Понтонный": 8603,
    "Усть-Ижора": 2617,
    "Петро-Славянка": 2146,
    "Саперный": 1733,
    "Металлострой": 27592,

    # Красногвардейский
    "Полюстрово": 76707,
    "Большая Охта": 58176,
    "Малая Охта": 43319,
    "Пороховые": 128807,
    "Ржевка": 61345,

    # Красносельский
    "Юго-Запад": 58269,
    "Южно-Приморский": 130109,
    "Сосновая поляна": 76863,
    "Урицк": 49984,
    "Константиновское": 35236,
    "Горелово": 29194,
    "Красное Село": 61162,

    # Кронштадт
    "Кронштадт": 44655,

    # Курортный
    "Зеленогорск": 15568,
    "Сестрорецк": 46304,
    "Белоостров": 2466,
    "Комарово": 1476,
    "Молодежное": 1588,
    "Песочный": 9977,
    "Репино": 3022,
    "Серово": 301,
    "Смолячково": 748,
    "Солнечное": 2048,
    "Ушково": 811,

    # Московский
    "Московская застава": 45013,
    "Гагаринское": 65614,
    "Новоизмайловское": 83286,
    "Пулковский меридиан": 45710,
    "Звездное": 94318,

    # Невский
    "Невская застава": 28441,
    "Ивановский": 27516,
    "Обуховский": 57556,
    "Рыбацкое": 71335,
    "Народный": 81210,
    "№ 54": 68326,
    "Невский округ": 70577,
    "Оккервиль": 62501,
    "Правобережный": 89418,

    # Петроградский
    "Введенский": 15799,
    "Кронверкское": 16881,
    "Посадский": 16154,
    "Аптекарский остров": 21092,
    "округ Петровский": 17932,
    "Чкаловское": 30355,

    # Петродворцовый
    "Ломоносов": 38477,
    "Петергоф": 81767,
    "Стрельна": 14651,

    # Приморский
    "Лахта-Ольгино": 15968,
    "№ 65": 194361,
    "Ланское": 70883,
    "Комендантский аэродром": 95484,
    "Озеро Долгое": 92572,
    "Юнтолово": 116908,
    "Коломяги": 122082,
    "Лисий Нос": 6596,

    # Пушкинский
    "Павловск": 18485,
    "Пушкин": 108969,
    "Шушары": 144901,
    "Александровская": 3221,
    "Тярлево": 1765,

    # Фрунзенский
    "Волковское": 61224,
    "№ 72": 67523,
    "Купчино": 55794,
    "Георгиевский": 91841,
    "Александровский": 62485,
    "Балканский": 73604,

    # Центральный
    "Дворцовый округ": 7288,
    "№ 78": 8959,
    "Литейный округ": 39786,
    "Смольнинское": 73545,
    "Лиговка-Ямская": 19523,
    "Владимирский округ": 49761,
}

In [298]:
# Преобразование в DataFrame
df = pd.DataFrame.from_dict(districts_pop, orient='index', columns=['population'])

# Сброс индекса для создания колонки с названиями районов
df = df.reset_index().rename(columns={'index': 'mun_district'})

# Добавим площадь
df['area'] = df['mun_district'].apply(
    lambda x: districts_area.get(x, None)
    )

df.head()

,mun_district,population,area
0,Коломна,33543,1.89
1,Сенной округ,20342,1.10
2,Адмиралтейский округ,23971,2.10
3,Семеновский,22109,1.66
4,Измайловское,32761,3.19


### Плотность всего населения

По муниципальным округам (чел/км2)

In [299]:
# Плотность всего населения по округам
result = df

result["pop_dense"] = result["population"] / result["area"]
result

,mun_district,population,area,pop_dense
0,Коломна,33543,1.89,17747.619048
1,Сенной округ,20342,1.10,18492.727273
2,Адмиралтейский округ,23971,2.10,11414.761905
3,Семеновский,22109,1.66,13318.674699
4,Измайловское,32761,3.19,10269.905956
...,...,...,...,...
106,№ 78,8959,1.31,6838.931298
107,Литейный округ,39786,2.36,16858.474576
108,Смольнинское,73545,5.44,13519.301471
109,Лиговка-Ямская,19523,3.30,5916.060606


### Средний возраст

По муниципальным округам

In [300]:
result['avg_age'] = result['mun_district'].apply(
    lambda x: districts_age.get(x, None)
    )

result

,mun_district,population,area,pop_dense,avg_age
0,Коломна,33543,1.89,17747.619048,42.44
1,Сенной округ,20342,1.10,18492.727273,42.44
2,Адмиралтейский округ,23971,2.10,11414.761905,42.44
3,Семеновский,22109,1.66,13318.674699,42.44
4,Измайловское,32761,3.19,10269.905956,42.44
...,...,...,...,...,...
106,№ 78,8959,1.31,6838.931298,43.95
107,Литейный округ,39786,2.36,16858.474576,43.95
108,Смольнинское,73545,5.44,13519.301471,43.95
109,Лиговка-Ямская,19523,3.30,5916.060606,43.95


## Деятельность предприятий

- Количество организаций по данным государственной регистрации - data_Y49010002_112_v20241111
- Количество индивидуальных предпринимателей по данным государственной регистрации - data_Y49010003_112_v20241111

### Товары собственного производства

Файл: data_Y48401011_112_v20241111

In [301]:
file_path = "drive/MyDrive/masters/geo-embeddings/data/self_goods_stats.csv"

df = pd.read_csv(file_path)
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,okved2,region_id,region_name,mun_level,mun_district,municipality,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,indicator_value,indicator_unit,indicator_period,comment
0,39,Деятельность предприятий,Y48401011,"Отгружено товаров собственного производства, в...",Всего по обследуемым видам экономической деяте...,1,Алтайский край,Муниципальное образование верхнего уровня,Залесовский муниципальный округ,Залесовский муниципальный округ,...,Муниципальный округ,01512000,Изменен тип и название,2022,2025,2022,244664.8,Тысяча рублей,Январь-декабрь,CD
1,39,Деятельность предприятий,Y48401011,"Отгружено товаров собственного производства, в...",Всего по обследуемым видам экономической деяте...,1,Алтайский край,Муниципальное образование верхнего уровня,Залесовский муниципальный округ,Залесовский муниципальный округ,...,Муниципальный округ,01512000,Изменен тип и название,2022,2025,2022,127158.5,Тысяча рублей,Январь-июнь,CD


In [302]:
# Фильтруем Москву и последний отчетный год
df = df[(df["region_name"] == "Санкт-Петербург")
            & (df["year"] == 2023)
            & (df["okved2"] == "Всего по обследуемым видам экономической деятельности")
            & (df["indicator_period"] == "Январь-декабрь")]

# Убираем хвостики у названий
patterns = ['Муниципальный округ ', 'Городской округ ', 'Поселение ',
            'Поселок ', 'Город ']
for pattern in patterns:
        df['mun_district'] = df['mun_district'].str.replace(pattern, '')

# Кол-во муниципалитетов, округов и прочего
mun_num = len(df["mun_district"].unique())

print(f"СПб: {mun_num}")

СПб: 107


<ipython-input-302-1308907516>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mun_district'] = df['mun_district'].str.replace(pattern, '')


In [303]:
# Готово!
df = df.rename(columns={"indicator_value": "self_goods_sold"})
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,okved2,region_id,region_name,mun_level,mun_district,municipality,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,self_goods_sold,indicator_unit,indicator_period,comment
49097,39,Деятельность предприятий,Y48401011,"Отгружено товаров собственного производства, в...",Всего по обследуемым видам экономической деяте...,40,Санкт-Петербург,Субъект РФ,Муниципальные образования города федерального ...,Муниципальные образования города федерального ...,...,Субъект РФ,CD,CD,CD,CD,2023,8.743522e+09,Тысяча рублей,Январь-декабрь,mun_type в исходных данных не соответствует oktmo
49148,39,Деятельность предприятий,Y48401011,"Отгружено товаров собственного производства, в...",Всего по обследуемым видам экономической деяте...,40,Санкт-Петербург,Муниципальное образование верхнего уровня,Коломна,Муниципальный округ Коломна,...,Внутригородская территория города федерального...,40301000,Без изменений,2010,2025,2023,4.488500e+07,Тысяча рублей,Январь-декабрь,CD


In [304]:
# Продолжаем составлять итоговый датасет
result = pd.merge(result, df[["mun_district", "self_goods_sold"]], on="mun_district", how="left")
result

,mun_district,population,area,pop_dense,avg_age,self_goods_sold
0,Коломна,33543,1.89,17747.619048,42.44,4.488500e+07
1,Сенной округ,20342,1.10,18492.727273,42.44,6.365571e+07
2,Адмиралтейский округ,23971,2.10,11414.761905,42.44,1.659061e+09
3,Семеновский,22109,1.66,13318.674699,42.44,7.641300e+07
4,Измайловское,32761,3.19,10269.905956,42.44,2.457805e+07
...,...,...,...,...,...,...
106,№ 78,8959,1.31,6838.931298,43.95,5.916639e+07
107,Литейный округ,39786,2.36,16858.474576,43.95,1.263859e+08
108,Смольнинское,73545,5.44,13519.301471,43.95,2.347940e+08
109,Лиговка-Ямская,19523,3.30,5916.060606,43.95,5.975880e+07


In [305]:
result[result["self_goods_sold"].isna()]

,mun_district,population,area,pop_dense,avg_age,self_goods_sold
35,Усть-Ижора,2617,4.15,630.602410,43.05,NaN
55,Комарово,1476,31.00,47.612903,42.78,NaN
59,Серово,301,9.78,30.777096,42.78,NaN
62,Ушково,811,25.00,32.440000,42.78,NaN
97,Александровская,3221,4.77,675.262055,39.07,NaN


In [306]:
median = result['self_goods_sold'].median()
result['self_goods_sold'] = result['self_goods_sold'].fillna(median)
result[result["self_goods_sold"].isna()]

,mun_district,population,area,pop_dense,avg_age,self_goods_sold


### Количество организаций

Файл: data_Y49010002_112_v20241111

In [307]:
file_path = "drive/MyDrive/masters/geo-embeddings/data/org_num_stats.csv"

df = pd.read_csv(file_path)
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,region_id,region_name,mun_level,mun_district,municipality,oktmo,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,indicator_value,indicator_unit,indicator_period,comment
0,39,Деятельность предприятий,Y49010003,Количество индивидуальных предпринимателей по ...,1,Алтайский край,Муниципальное образование верхнего уровня,Залесовский муниципальный округ,Залесовский муниципальный округ,1512000,...,Муниципальный округ,01512000,Изменен тип и название,2022,2025,2024,217,Человек,На 1 апреля,CD
1,39,Деятельность предприятий,Y49010003,Количество индивидуальных предпринимателей по ...,1,Алтайский край,Муниципальное образование верхнего уровня,Муниципальный округ город Славгород,Муниципальный округ город Славгород,1539000,...,Муниципальный округ,01539000,"Изменен тип и название, Объединение",2023,2025,2024,854,Человек,На 1 апреля,CD


In [308]:
# Фильтруем
df = df[df["region_name"] == "Санкт-Петербург"]

# Убираем хвостики у названий
patterns = ['Муниципальный округ ', 'Городской округ ', 'Поселение ',
            'Поселок ', 'Город ']
for pattern in patterns:
        df['mun_district'] = df['mun_district'].str.replace(pattern, '')

# Кол-во муниципалитетов, округов и прочего
mun_num = len(df["mun_district"].unique())

print(f"СПб: {mun_num}")

СПб: 112


<ipython-input-308-2456129796>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mun_district'] = df['mun_district'].str.replace(pattern, '')


In [309]:
# Готово!
df = df.rename(columns={"indicator_value": "org_num"})
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,region_id,region_name,mun_level,mun_district,municipality,oktmo,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,org_num,indicator_unit,indicator_period,comment
5957,39,Деятельность предприятий,Y49010003,Количество индивидуальных предпринимателей по ...,40,Санкт-Петербург,Субъект РФ,Муниципальные образования города федерального ...,Муниципальные образования города федерального ...,40000000,...,Субъект РФ,CD,CD,CD,CD,2024,211954,Человек,На 1 апреля,mun_type в исходных данных не соответствует oktmo
5958,39,Деятельность предприятий,Y49010003,Количество индивидуальных предпринимателей по ...,40,Санкт-Петербург,Муниципальное образование верхнего уровня,Коломна,Муниципальный округ Коломна,40301000,...,Внутригородская территория города федерального...,40301000,Без изменений,2010,2025,2024,1279,Человек,На 1 апреля,CD


In [311]:
# Продолжаем составлять итоговый датасет
result = pd.merge(result, df[["mun_district", "org_num"]], on="mun_district", how="left")
result

,mun_district,population,area,pop_dense,avg_age,self_goods_sold,org_num
0,Коломна,33543,1.89,17747.619048,42.44,4.488500e+07,1279
1,Сенной округ,20342,1.10,18492.727273,42.44,6.365571e+07,766
2,Адмиралтейский округ,23971,2.10,11414.761905,42.44,1.659061e+09,944
3,Семеновский,22109,1.66,13318.674699,42.44,7.641300e+07,912
4,Измайловское,32761,3.19,10269.905956,42.44,2.457805e+07,1658
...,...,...,...,...,...,...,...
106,№ 78,8959,1.31,6838.931298,43.95,5.916639e+07,454
107,Литейный округ,39786,2.36,16858.474576,43.95,1.263859e+08,2167
108,Смольнинское,73545,5.44,13519.301471,43.95,2.347940e+08,3828
109,Лиговка-Ямская,19523,3.30,5916.060606,43.95,5.975880e+07,965


### Количество ИП

Файл: data_Y49010003_112_v20241111

In [312]:
file_path = "drive/MyDrive/masters/geo-embeddings/data/bus_num_stats.csv"

df = pd.read_csv(file_path)
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,region_id,region_name,mun_level,mun_district,municipality,oktmo,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,indicator_value,indicator_unit,indicator_period,comment
0,39,Деятельность предприятий,Y49010002,Количество организаций по данным государственн...,1,Алтайский край,Муниципальное образование верхнего уровня,Залесовский муниципальный округ,Залесовский муниципальный округ,1512000,...,Муниципальный округ,01512000,Изменен тип и название,2022,2025,2024,75,Единица,На 1 апреля,CD
1,39,Деятельность предприятий,Y49010002,Количество организаций по данным государственн...,1,Алтайский край,Муниципальное образование верхнего уровня,Муниципальный округ город Славгород,Муниципальный округ город Славгород,1539000,...,Муниципальный округ,01539000,"Изменен тип и название, Объединение",2023,2025,2024,266,Единица,На 1 апреля,CD


In [313]:
# Фильтруем Москву
df = df[df["region_name"] == "Санкт-Петербург"]

# Убираем хвостики у названий
patterns = ['Муниципальный округ ', 'Городской округ ', 'Поселение ',
            'Поселок ', 'Город ']
for pattern in patterns:
        df['mun_district'] = df['mun_district'].str.replace(pattern, '')

# Кол-во муниципалитетов, округов и прочего
mun_num = len(df["mun_district"].unique())

print(f"СПб: {mun_num}")

<ipython-input-313-894397923>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mun_district'] = df['mun_district'].str.replace(pattern, '')


СПб: 112


In [314]:
# Готово!
df = df.rename(columns={"indicator_value": "entrep_num"})
df.head(2)

,indicator_section_code,indicator_section,indicator_code,indicator_name,region_id,region_name,mun_level,mun_district,municipality,oktmo,...,mun_type_oktmo,oktmo_stable,oktmo_history,oktmo_year_from,oktmo_year_to,year,entrep_num,indicator_unit,indicator_period,comment
5995,39,Деятельность предприятий,Y49010002,Количество организаций по данным государственн...,40,Санкт-Петербург,Субъект РФ,Муниципальные образования города федерального ...,Муниципальные образования города федерального ...,40000000,...,Субъект РФ,CD,CD,CD,CD,2024,208461,Единица,На 1 апреля,mun_type в исходных данных не соответствует oktmo
5996,39,Деятельность предприятий,Y49010002,Количество организаций по данным государственн...,40,Санкт-Петербург,Муниципальное образование верхнего уровня,Коломна,Муниципальный округ Коломна,40301000,...,Внутригородская территория города федерального...,40301000,Без изменений,2010,2025,2024,1589,Единица,На 1 апреля,CD


In [315]:
# Продолжаем составлять итоговый датасет
result = pd.merge(result, df[["mun_district", "entrep_num"]], on="mun_district", how="left")
result

,mun_district,population,area,pop_dense,avg_age,self_goods_sold,org_num,entrep_num
0,Коломна,33543,1.89,17747.619048,42.44,4.488500e+07,1279,1589
1,Сенной округ,20342,1.10,18492.727273,42.44,6.365571e+07,766,1811
2,Адмиралтейский округ,23971,2.10,11414.761905,42.44,1.659061e+09,944,1777
3,Семеновский,22109,1.66,13318.674699,42.44,7.641300e+07,912,1517
4,Измайловское,32761,3.19,10269.905956,42.44,2.457805e+07,1658,2734
...,...,...,...,...,...,...,...,...
106,№ 78,8959,1.31,6838.931298,43.95,5.916639e+07,454,1559
107,Литейный округ,39786,2.36,16858.474576,43.95,1.263859e+08,2167,3991
108,Смольнинское,73545,5.44,13519.301471,43.95,2.347940e+08,3828,7398
109,Лиговка-Ямская,19523,3.30,5916.060606,43.95,5.975880e+07,965,3055


In [316]:
!wget -nc https://download.geofabrik.de/russia/northwestern-fed-district-latest.osm.pbf

File ‘northwestern-fed-district-latest.osm.pbf’ already there; not retrieving.



In [317]:
overpass_url = "https://overpass-api.de/api/interpreter"
query = """
[out:json][timeout:25];
relation["boundary"="administrative"]["admin_level"="4"]["name"="Санкт-Петербург"];
out geom;
"""

resp = requests.get(overpass_url, params={"data": query})
data = resp.json()

In [318]:
# Создаем список всех полигонов районов
district_polygons = []
for element in data["elements"]:
    if element["type"] == "relation":
        members = element.get("members", [])
        lines = []
        for m in members:
            if m["role"] in ["outer", ""] and "geometry" in m:  # Некоторые роли могут быть пустыми
                coords = [(pt["lon"], pt["lat"]) for pt in m["geometry"]]
                if len(coords) > 1:  # Проверяем, что линия валидна
                    lines.append(LineString(coords))

        if lines:
            try:
                # Пробуем создать полигон из линий
                polygons = list(polygonize(lines))
                if polygons:
                    district_polygon = unary_union(polygons)
                    district_polygons.append({
                        "type": "Feature",
                        "properties": {"name": element["tags"].get("name", "")},
                        "geometry": json.loads(json.dumps(district_polygon.__geo_interface__))
                    })
            except:
                print(f"Ошибка при обработке района {element.get('id')}")

# Сохраняем все районы как FeatureCollection
geojson = {
    "type": "FeatureCollection",
    "features": district_polygons
}

with open("spb_districts.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print("Границы районов сохранены в spb_districts.geojson")

Границы районов сохранены в spb_districts.geojson


In [319]:
!osmium extract \
    --overwrite \
    --output-format pbf \
    --polygon spb_districts.geojson \
    northwestern-fed-district-latest.osm.pbf \
    -o spb-districts.osm.pbf

[======================================================================] 100% 


In [320]:
fp = "spb-districts.osm.pbf"

osm = OSM(fp)

# Получаем все административные границы
boundaries = osm.get_boundaries()

In [321]:
print(boundaries.keys()), len(boundaries)

Index(['addr:country', 'addr:postcode', 'name', 'ref', 'url', 'website',
       'admin_level', 'boundary', 'id', 'timestamp', 'version', 'changeset',
       'geometry', 'tags', 'osm_type'],
      dtype='object')


(None, 159)

In [322]:
mun_geo = boundaries[["name", "geometry"]]
# mun_geo.to_csv("mun_geo_spb.csv")
mun_geo = pd.read_csv("mun_geo_spb.csv")

In [323]:
mun_geo = mun_geo.rename(columns={"name": "mun_district"})
mun_geo['mun_district'] = mun_geo['mun_district'].str.replace("ё", 'е')

patterns = ['городской округ ', 'район ', ' район', 'округ ',
            'муниципальный округ', ' сельское поселение',
            ' городское поселение', ' округ']
# ['Муниципальный округ ', 'Городской округ ', 'Поселение ',
#             'Поселок ', 'Город ']
for pattern in patterns:
        mun_geo['mun_district'] = mun_geo['mun_district'].str.replace(pattern, '')

mun_geo.head()

,Unnamed: 0,mun_district,geometry
0,0,Ленинградская область,MULTIPOLYGON (((30.650354385375977 59.72186660...
1,1,Санкт-Петербург,POLYGON ((29.438899993896484 60.17642974853515...
2,2,Колпино,POLYGON ((30.50885772705078 59.760658264160156...
3,3,Колпинский,POLYGON ((30.441120147705078 59.82937240600586...
4,4,Понтонный,POLYGON ((30.60064125061035 59.793724060058594...


In [324]:
mun_geo["mun_district"].unique()

array(['Ленинградская область', 'Санкт-Петербург', 'Колпино',
       'Колпинский', 'Понтонный', 'Саперный', 'Металлострой',
       'Усть-Ижора', 'Петро-Славянка', 'Пушкинский', 'Московский',
       'Московская застава', 'Шушары', 'Александровская', 'Тярлево',
       'Пушкин', 'Павловск', 'Красное Село', 'Горелово', 'Красносельский',
       'Южно-Приморский', 'Сосновая поляна', 'Урицк', 'Константиновское',
       'Юго-Запад', 'Петродворцовый', 'Стрельна', 'Петергоф', 'Ломоносов',
       'Невский', 'Невская застава', 'Народный', '№ 54', 'Обуховский',
       'Рыбацкое', 'Ивановский', 'Фрунзенский', 'Правобережный',
       'Оккервиль', 'Ломоносовский', 'Полянское', 'Адмиралтейский',
       'Василеостровский', 'Выборгский', 'Калининский', 'Кировский',
       'Красногвардейский', 'Центральный', 'Петроградский',
       'Кронштадтский', 'Курортный', 'Приморский', 'Сестрорецк',
       'Песочный', 'Белоостров', 'Солнечное', 'Первомайское', 'Репино',
       'Комарово', 'Ушково', 'Рощинское', 'Зел

In [325]:
replacements = {
    'Сенной': 'Сенной округ',
    'Адмиралтейский': 'Адмиралтейский округ',
    'Финляндский': 'Финляндский округ',
    'Нарвский': 'Нарвский округ',
    'Невский': 'Невский округ',
    'Дворцовый': 'Дворцовый округ',
    'Литейный': 'Литейный округ',
    'Владимирский': 'Владимирский округ',
}
mun_geo['mun_district'] = mun_geo['mun_district'].replace(replacements)

In [326]:
# mun_geo.to_csv("mun_geo_spb.csv")

In [327]:
# Добавляем координаты в итоговый датасет
final = pd.merge(result, mun_geo[["mun_district", "geometry"]], on="mun_district", how="left")
# final = final.drop("indicator_value", axis=1)
final

,mun_district,population,area,pop_dense,avg_age,self_goods_sold,org_num,entrep_num,geometry
0,Коломна,33543,1.89,17747.619048,42.44,4.488500e+07,1279,1589,"POLYGON ((30.2604923248291 59.917259216308594,..."
1,Сенной округ,20342,1.10,18492.727273,42.44,6.365571e+07,766,1811,POLYGON ((30.299034118652344 59.92062377929687...
2,Адмиралтейский округ,23971,2.10,11414.761905,42.44,1.659061e+09,944,1777,POLYGON ((30.250228881835938 59.90129089355469...
3,Адмиралтейский округ,23971,2.10,11414.761905,42.44,1.659061e+09,944,1777,POLYGON ((30.281797409057617 59.93306350708008...
4,Семеновский,22109,1.66,13318.674699,42.44,7.641300e+07,912,1517,POLYGON ((30.317771911621094 59.92169570922851...
...,...,...,...,...,...,...,...,...,...
108,№ 78,8959,1.31,6838.931298,43.95,5.916639e+07,454,1559,POLYGON ((30.310129165649414 59.93646621704101...
109,Литейный округ,39786,2.36,16858.474576,43.95,1.263859e+08,2167,3991,POLYGON ((30.334888458251953 59.95015335083008...
110,Смольнинское,73545,5.44,13519.301471,43.95,2.347940e+08,3828,7398,"POLYGON ((30.359195709228516 59.9440803527832,..."
111,Лиговка-Ямская,19523,3.30,5916.060606,43.95,5.975880e+07,965,3055,POLYGON ((30.350631713867188 59.91523742675781...


In [328]:
# final[final["geometry"].isna()]
final.to_csv("spb_mun_geo_params.csv")